In [1]:
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import InceptionV3

import pandas as pd

print("tensorflow version: " + str(tf.__version__))
print("pandas version: " + str(pd.__version__))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TF_CUDNN_WORKSPACE_LIMIT_IN_MB = 0

tensorflow version: 2.6.0
pandas version: 1.1.3
Num GPUs Available:  1


In [2]:
inception = InceptionV3()

pmodel_in = inception.get_layer(index = 0).input
pmodel_out = inception.get_layer(index = -2).output

print("Output layer of the first part of the model:\n", pmodel_out)

pmodel = Model(inputs = pmodel_in, outputs = pmodel_out)

for layer in pmodel.layers:
    layer.trainable = False

Output layer of the first part of the model:
 KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='avg_pool/Mean:0', description="created by layer 'avg_pool'")


In [3]:
model = Sequential()

model.add(pmodel)
model.add(Dense(units = 2, activation = "sigmoid", input_dim = 2048))

model.compile(optimizer = "Adam",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________
None


In [5]:
data_path = "S:\\source\\image-generator\\out"

df = pd.read_csv(os.path.join(data_path, "index0.csv"))
labels = ["pass", "fail"]
gen = ImageDataGenerator()

training_data = gen.flow_from_dataframe(dataframe = df[:850],
                                        directory = os.path.join(data_path, "imgs"),
                                        x_col = "Filenames",
                                        y_col = labels,
                                        batch_size = 16,
                                        shuffle = True,
                                        class_mode = "raw",
                                        target_size = (640,480))

validation_data = gen.flow_from_dataframe(dataframe = df[850:1100],
                                          directory = os.path.join(data_path, "imgs"),
                                          x_col = "Filenames",
                                          y_col = labels,
                                          batch_size = 16,
                                          shuffle = True,
                                          class_mode = "raw",
                                          target_size = (640,480))


test_data = gen.flow_from_dataframe(dataframe = df[1100:],
                                    directory = os.path.join(data_path, "imgs"),
                                    x_col = "Filenames",
                                    batch_size = 1,
                                    shuffle = True,
                                    class_mode = None,
                                    target_size = (640,480))

Found 849 validated image filenames.
Found 250 validated image filenames.
Found 100 validated image filenames.


In [6]:
model.fit(training_data, validation_data = validation_data, epochs = 10)

Epoch 1/10
54/54 [==============================] - 105s 2s/step - loss: 1.1914 - accuracy: 0.6478 - val_loss: 1.2921 - val_accuracy: 0.6600
Epoch 2/10
54/54 [==============================] - 79s 1s/step - loss: 1.2338 - accuracy: 0.6372 - val_loss: 0.9226 - val_accuracy: 0.5440
Epoch 3/10
54/54 [==============================] - 79s 1s/step - loss: 0.8940 - accuracy: 0.6572 - val_loss: 0.8981 - val_accuracy: 0.5640
Epoch 4/10
54/54 [==============================] - 79s 1s/step - loss: 0.8480 - accuracy: 0.6631 - val_loss: 0.7869 - val_accuracy: 0.6520
Epoch 5/10
54/54 [==============================] - 79s 1s/step - loss: 0.9150 - accuracy: 0.6549 - val_loss: 0.8282 - val_accuracy: 0.6520
Epoch 6/10
54/54 [==============================] - 79s 1s/step - loss: 0.7743 - accuracy: 0.6702 - val_loss: 0.7640 - val_accuracy: 0.6320
Epoch 7/10
54/54 [==============================] - 79s 1s/step - loss: 0.6638 - accuracy: 0.6784 - val_loss: 0.8449 - val_accuracy: 0.6760
Epoch 8/10
54/54 [=

In [7]:
model.save("./model")

INFO:tensorflow:Assets written to: ./model\assets
